# LangChain RAG 201

Tentative techniques to explore:

1. MultiQueryRetriever
2. Contextual Compression
3. Ensemble Retrievers
4. Self-quering Retrievers
5. Time weighted vector store retrievers

In [2]:
%pip install --q langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest 7.2.1 requires attrs>=19.2.0, but you have attrs 18.2.0 which is incompatible.
gradient 2.0.6 requires marshmallow<3.0, but you have marshmallow 3.20.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [16]:
# %> curl -fsSL https://ollama.com/install.sh | sh
# %> ollama serve
# %> ollama pull llama2
# %> ollama pull nomic-embed-text

!ollama list

NAME         	ID          	SIZE  	MODIFIED       
llama2:latest	78e26419b446	3.8 GB	17 minutes ago	


In [8]:
from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    temperature=0.9,
)

llm("Tell me 5 facts about Roman history")


 Of course! Here are five interesting facts about Roman history:

1. The Roman Empire was the largest empire in the world at its peak, spanning across three continents and encompassing over 20% of the world's land area.
2. The Roman language, Latin, is still studied today and has had a profound influence on many modern languages, including English. In fact, over 75% of English words have roots in Latin.
3. The Roman calendar, which was based on a solar year, was the precursor to the modern Gregorian calendar used today. The Romans also invented the concept of days of the week, with names like Sunday (Latin: Dies Solis) and Monday (Latin: Dies Lunae) still in use today.
4. The Roman Colosseum, completed in 80 AD, was the largest amphitheater in the ancient world and could hold up to 50,000 spectators. It was used for gladiatorial contests, animal hunts, and other public events.
5. The Roman Empire had a complex system of governance, with power divided between the emperor, the Senate, a

"\n Of course! Here are five interesting facts about Roman history:\n\n1. The Roman Empire was the largest empire in the world at its peak, spanning across three continents and encompassing over 20% of the world's land area.\n2. The Roman language, Latin, is still studied today and has had a profound influence on many modern languages, including English. In fact, over 75% of English words have roots in Latin.\n3. The Roman calendar, which was based on a solar year, was the precursor to the modern Gregorian calendar used today. The Romans also invented the concept of days of the week, with names like Sunday (Latin: Dies Solis) and Monday (Latin: Dies Lunae) still in use today.\n4. The Roman Colosseum, completed in 80 AD, was the largest amphitheater in the ancient world and could hold up to 50,000 spectators. It was used for gladiatorial contests, animal hunts, and other public events.\n5. The Roman Empire had a complex system of governance, with power divided between the emperor, the S

In [17]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")
text = "Embed this text"
embed = embeddings.embed_query(text)
embed

ValueError: Error raised by inference API HTTP code: 404, {"error":"model 'nomic-embed-text' not found, try pulling it first"}

In [ ]:
text = "This is a test document."

To generate embeddings, you can either query an invidivual text, or you can query a list of texts.

query_result = embeddings.embed_query(text)
query_result[:5]